In [71]:
import tensorflow as tf
import face_detection
import dlib
from fer import FER
import cv2
import os
from mtcnn import MTCNN
import keras
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import torch
import gc
from datetime import datetime
import time

%matplotlib inline

In [2]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


31-07-2020:01:59:56,445 WARNING  [deprecation.py:323] From <ipython-input-2-2e82a26757ae>:2: is_gpu_available (from tensorflow.python.framework.test_util) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False


In [3]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [78]:
# traing path 
cur_path = os.path.abspath(os.getcwd())
img_dir = os.path.join(cur_path, "train\\img")
print(img_dir)
result_dir = os.path.join(cur_path, "train\\img_result")

img_list = os.listdir(img_dir) 
print(img_list[:10])


C:\Users\parkh\Desktop\[NOTA]facial_emotion_data\train\img
['1.1.jpg', '1.image_8640454461499434291050.jpg', '1.jjCU.jpg', '10.10.jpg', '10.6fb6616b7bf78bdb_liu.jpg', '10.jj1tn.jpg', '100.smile-asians-2.jpg', '100.VD34944438_w640.jpg', '101.EliKiseopUKISS.jpg', '101.SSI_20161018150057.jpg']


In [87]:

def run(start, end):  
    # emotion detector
    emotion_detector = FER()

    # face detector
    face_detector = face_detection.build_detector(
      "DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)
    
    cnt = 0
    for img_name in img_list[start:end]:  
            
        # img
        img_path = os.path.join(img_dir, img_name)

        # BGR to RGB
#         image = cv2.imread(img_path)[:, :, ::-1]
        image = cv2.imread(img_path)

        # face position
        detections = face_detector.detect(image)

        # show origin
        print("<",img_name, ">")
#         plt.imshow(image,aspect="auto")
#         plt.show()
        if len(detections) == 0 :
            cv2.imwrite(result_dir+"\\failed\\"+img_name,crop_image)  
            print("detecting failed !!")
        else:
#             print("detecting sucessed !!")
            pass



        for i in range(len(detections)):
            margin = 40
            top = int(round(detections[i][1]))-margin
            top = max(0, top)
            bottom = int(round(detections[i][3]))+margin
            left = int(round(detections[i][0]))-margin
            left = max(0,left)
            right =  int(round(detections[i][2]))+margin

            crop_image = image[top:bottom, left:right]
    #         print(crop_image)
    #         plt.imshow(crop_image,aspect="auto")
    #         plt.show()
    #         p = detections

            # show window
    #         win = dlib.image_window()
    #         win.set_image(im)

            # emotion 

            detect_res = emotion_detector.detect_emotions(crop_image)
            save_name = img_name[:-4] +"_"+ str(i) +".jpg"
            
            if len(detect_res):
                maximum = 0
                top_emotion = ''
                for emotion in detect_res[0]['emotions']:
                    prop = detect_res[0]['emotions'][emotion]
                    if maximum < prop:
                        maximum = prop
                        top_emotion = emotion
                
                if top_emotion == 'fear':
                    top_emotion = 'surprise'
                if top_emotion == 'diguste':
                    top_emotion = 'anger'
                
                cv2.imwrite(result_dir+"\\"+top_emotion+"\\"+save_name,crop_image)
                print("emotion is ", top_emotion)

                del maximum
                del top_emotion
                
            else:
                print("emotion is failed")
                cv2.imwrite(result_dir+"\\failed\\"+save_name,crop_image)
            
            del crop_image
            del detect_res
               
        del detections
        del image
        del img_path
        
    del emotion_detector
    del face_detector
    
    


In [88]:
batch_size = 1
epoch = 0
start = 33
end = len(img_list)

start_time = time.time()
print("batch size: ", batch_size, "total epoch:", end-start)
for i in range(start,end,batch_size):
    epoch += batch_size
    print("\nepoch: ", epoch)
    torch.cuda.empty_cache()
    gc.collect()
    run(i,i+1)
    
end_time = datetime.time()
print("elapsed time: ", end_time-start_time)


batch size:  1 total epoch: 1655

epoch:  1
< 11.6.jpg >
emotion is  happy

epoch:  2
< 11.asian-man-with-surprised-shocked-face_39688-1297.jpg >
emotion is  surprise

epoch:  3
< 11.happy-asian-family-stock-photo-2726146.jpg >
emotion is  happy
emotion is  happy
emotion is  happy
emotion is  happy

epoch:  4
< 11.VD57983418_w1280.jpg >
emotion is failed

epoch:  5
< 110.1129815437.jpg >
emotion is  surprise

epoch:  6
< 110.b2d812ad64e07994fe688c529274bb00.jpg >
emotion is  sad

epoch:  7
< 111.280-jir0032-jj.jpg >
emotion is  happy

epoch:  8
< 111.Asian-girls-shopping-515.jpg >
emotion is failed
emotion is failed
emotion is failed
emotion is failed
emotion is failed
emotion is failed
emotion is failed
emotion is failed

epoch:  9
< 111.crop.JPG >
emotion is  happy

epoch:  10
< 111.crop2.JPG >
emotion is  sad

epoch:  11
< 111.crop3.JPG >
emotion is failed

epoch:  12
< 111.crop4.JPG >
emotion is  sad

epoch:  13
< 111.crop5.JPG >
emotion is  neutral

epoch:  14
< 111.maxresdefault.

RuntimeError: CUDA out of memory. Tried to allocate 1.32 GiB (GPU 0; 8.00 GiB total capacity; 4.26 GiB already allocated; 807.19 MiB free; 5.12 GiB reserved in total by PyTorch)